In [ ]:
import os
from google import genai
from google.genai.errors import APIError

def test_gemini_api_key():
    """
    Khởi tạo và kiểm tra API key của Gemini bằng một truy vấn đơn giản.
    """
    try:

        api_key = "" 
        
        if not api_key or api_key == "YOUR_API_KEY_CỦA_BẠN":
            print("LỖI: Bạn chưa thay thế placeholder 'YOUR_API_KEY_CỦA_BẠN' bằng khóa API thực tế.")
            print("Vui lòng thay thế hoặc đặt biến môi trường 'GEMINI_API_KEY'.")
            return


        client = genai.Client(api_key=api_key) 

        # Câu hỏi đơn giản để kiểm tra
        question = "Thủ đô của nước Pháp là gì?"
        print(f"Đang gửi truy vấn: '{question}'")

        # Gọi API để tạo nội dung
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=question
        )


        print("\n--- KẾT QUẢ KIỂM TRA API ---")
        print(f"Trạng thái: THÀNH CÔNG 🎉")
        print(f"Phản hồi từ Gemini: {response.text.strip()}")
        print("---------------------------")
        print("\nAPI key đã hoạt động.")

    except APIError as e:
        print("\n--- LỖI KIỂM TRA API ---")
        print(f"Trạng thái: THẤT BẠI ❌")
        print(f"Lỗi API: {e}")
        print("Có thể API key không hợp lệ, bị thu hồi hoặc đã vượt quá hạn mức sử dụng.")
        print("------------------------")
    except Exception as e:
        print(f"\nMột lỗi không mong muốn đã xảy ra: {e}")

if __name__ == "__main__":
    test_gemini_api_key()

Đang gửi truy vấn: 'Thủ đô của nước Pháp là gì?'

--- KẾT QUẢ KIỂM TRA API ---
Trạng thái: THÀNH CÔNG 🎉
Phản hồi từ Gemini: Thủ đô của nước Pháp là **Paris**.
---------------------------

API key đã hoạt động.


In [ ]:
import re
import os

# --- CÁC THAM SỐ CẤU HÌNH ---
# Kích thước tối đa mong muốn của mỗi chunk (tính bằng KÝ TỰ). 
# Tối ưu cho Gemini Flash.
MAX_CHUNK_SIZE = 20000 
FILE_PATH = r'E:\HistoryChatbot\backend\src\output.txt' 

def read_file(file_path):
    """Đọc toàn bộ nội dung tệp và chuẩn hóa xuống dòng."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Không tìm thấy tệp: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    content = content.replace('\r\n', '\n').replace('\r', '\n')
    content = re.sub(r'\n{3,}', '\n\n', content) 
    return content

def smart_chunking(text, max_size):
    """
    Chia văn bản thành các chunk KHÔNG có overlap.
    Ưu tiên ngắt tại dấu chấm câu/ngắt đoạn để giữ ngữ cảnh.
    """
    chunks = []
    current_pos = 0
    
    while current_pos < len(text):
        end_pos = min(current_pos + max_size, len(text))
        
        if end_pos == len(text):
            chunk = text[current_pos:end_pos]
            chunks.append(chunk.strip())
            break
        
        # Tìm điểm ngắt tự nhiên gần nhất (dấu chấm câu hoặc ngắt đoạn)
        # Tìm từ cuối lên, trong phạm vi an toàn (khoảng 80% kích thước chunk)
        search_start = max(current_pos, end_pos - max_size // 5) 
        
        # Sử dụng regex tìm kiếm dấu chấm câu theo sau bởi khoảng trắng/xuống dòng
        split_match = re.search(r'[.?!]\s*\n*\s*', text[search_start:end_pos][::-1])
        
        if split_match:
            # Vị trí ngắt tuyệt đối
            relative_split_pos = end_pos - (search_start + split_match.start())
            split_point = search_start + relative_split_pos
            
            chunk = text[current_pos:split_point].strip()
            chunks.append(chunk)
            
            # Cập nhật vị trí bắt đầu cho chunk tiếp theo
            current_pos = split_point 
        else:
            # Nếu không tìm thấy dấu ngắt tự nhiên, buộc phải ngắt tại end_pos
            chunk = text[current_pos:end_pos].strip()
            chunks.append(chunk)
            current_pos = end_pos
            
    return chunks

try:
    full_text = read_file(FILE_PATH)
    
    # Chạy hàm chia chunk
    text_chunks = smart_chunking(full_text, MAX_CHUNK_SIZE)
    
    # Hiển thị kết quả
    print(f"Tệp đã được đọc thành công ({len(full_text)} ký tự).")
    print(f"Đã chia thành {len(text_chunks)} chunk để gửi lên Gemini API.")
    print(f"Kích thước trung bình của chunk: {sum(len(c) for c in text_chunks) / len(text_chunks):.0f} ký tự.")
    print("\n--- 3 CHUNK ĐẦU TIÊN (VÍ DỤ) ---")
    for i, chunk in enumerate(text_chunks[:3]):
        print(f"\n[CHUNK #{i+1}, {len(chunk)} ký tự]")
        print(chunk[:500] + ('...' if len(chunk) > 500 else ''))

except FileNotFoundError as e:
    print(f"LỖI: {e}. Vui lòng kiểm tra lại đường dẫn tệp.")
except Exception as e:
    print(f"Đã xảy ra lỗi không xác định: {e}")


✅ Tệp đã được đọc thành công (887242 ký tự).
✅ Đã chia thành 45 chunk để gửi lên Gemini API.
Kích thước trung bình của chunk: 19715 ký tự.

--- 3 CHUNK ĐẦU TIÊN (VÍ DỤ) ---

[CHUNK #1, 19987 ký tự]
LỜI NÓI ĐÀU 

Thắng lợi của cuộc kháng chiến chống thực dân Pháp với đỉnh  cao là chiến dịch Điện Biên Phù đã dẫn tới việc ký kết Hiệp định  €iiơnevơ vào ngày 20 tháng 7 năm 1954 lập lại hòa bình trên toàn  côi Dông Dương trên cơ sở tôn trọng độc lập, chủ quyền, thống nhất  và toàn vẹn lãnh thô của Việt Nam, Lào và Campuchia. Riêng đối  với Việt Nam. Hiệp định nêu rõ: Hai năm sau ngày ký Hiệp định. tức  khoảng tháng 7 năm 1956, nước Việt Nam sẽ tô chức Tông tuyển cử  tự do trong cả nước nhằm hòa...

[CHUNK #2, 19850 ký tự]
Thực ra, âm mưu trên đây của Mỹ đã có từ trước. Ngay từ khi  vấn đề về chấm dứt chiến tranh ở Việt Nam và Đông Dương còn  đang đàm phán. Tông thống Mỹ Aixenhao (Eisenhower) đã lớn  tiếng hô hào: "Nếu Hội nghị Giơnevơ đi đến ký kết, sẽ tổ chức một  cuộc đi cư có kế hoạch ng

In [3]:
text_chunks[0]

'LỜI NÓI ĐÀU \n\nThắng lợi của cuộc kháng chiến chống thực dân Pháp với đỉnh  cao là chiến dịch Điện Biên Phù đã dẫn tới việc ký kết Hiệp định  €iiơnevơ vào ngày 20 tháng 7 năm 1954 lập lại hòa bình trên toàn  côi Dông Dương trên cơ sở tôn trọng độc lập, chủ quyền, thống nhất  và toàn vẹn lãnh thô của Việt Nam, Lào và Campuchia. Riêng đối  với Việt Nam. Hiệp định nêu rõ: Hai năm sau ngày ký Hiệp định. tức  khoảng tháng 7 năm 1956, nước Việt Nam sẽ tô chức Tông tuyển cử  tự do trong cả nước nhằm hòa bình thông nhất đất nước. Trong khi  chờ tiền hành tông tuyên cử. hai bên ngừng bắn. chuyền quân tập  kết vẻ hai miền Nam, Bắc, lấy vĩ tuyển 17 làm vĩ tuyến quân sự  tạm thời. \n\nChính phủ Việt Nam Dân chủ Cộng hòa đã nghiêm chỉnh cấp  hành các điều khoản được nêu trong Hiệp định Giơnevơ: Tập kết  quân đội về các nơi quy định rồi từ đó lên tàu ra Bắc. Chỉ hai ngày  sau khi ký Hiệp định Giơnevơ, Chủ tịch Hồ Chí Minh xác định:  "Chúng ta phải ra sức đấu tranh dẻ thực hiện tổng tuyển cử tự do 

In [ ]:
import google.genai as genai
import time
import os
import re

# ==============================================================================
#                  CẤU HÌNH & KHỞI TẠO
# ==============================================================================
# --- THÔNG SỐ CẤU HÌNH ---
API_KEY = "" 
MODEL_NAME = 'gemini-2.5-flash' 
DELAY_SECONDS = 5                 
OUTPUT_FILENAME = 'tai_lieu_da_sua_loi_final.md'

# Khởi tạo Client
try:
    client = genai.Client(api_key=API_KEY)
except Exception as e:
    print(f"Lỗi khởi tạo Gemini Client: {e}. Vui lòng kiểm tra lại API Key.")
    client = None


# ==============================================================================
#                  GỌI API
# ==============================================================================

def correct_ocr_error(chunk_text, client):
    """Gửi một đoạn văn bản (chunk) tới Gemini để sửa lỗi OCR."""

    # 1. System Instruction (ĐÃ SỬA: Tách ra khỏi contents và tích hợp yêu cầu mới)
    system_instruction = (
        "Bạn là chuyên gia sửa lỗi chính tả và phục hồi định dạng từ văn bản OCR Tiếng Việt. "
        "Văn bản đầu vào bị lỗi. "
        "Yêu cầu của bạn là: "
        "1. Sửa tất cả lỗi chính tả, ngữ pháp, và dấu câu Tiếng Việt. "
        "2. Phục hồi đúng định dạng, **sử dụng Markdown để làm nổi bật các tiêu đề chính như Chương, Section và subsection để phân biệt rõ chúng với các liệt kê trong văn bản (sử dụng `# Title`)**. "
        "3. **Xóa hoàn toàn các phần chú thích sách thừa thãi** (ví dụ: số trang, tên sách lặp lại ở đầu/cuối trang hoặc các trích dẫn đến sách khác) và giữ nguyên nội dung chính. "
        "4. Tuyệt đối **KHÔNG** thêm bất kỳ lời giải thích, tiêu đề mới, hoặc nội dung mới nào. "
        "Chỉ trả về văn bản đã sửa và đã được làm sạch dưới dạng Markdown."
    )
    
    # 2. Contents (ĐÃ SỬA: Chỉ chứa role 'user')
    contents = [
        {"parts": [{"text": f"Văn bản cần sửa lỗi OCR: {chunk_text}"}]}
    ]
    
    # 3. Generation Config (ĐÃ SỬA: Dùng để truyền System Instruction)
    generation_config = genai.types.GenerateContentConfig(
        system_instruction=system_instruction
    )
    
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=contents, # Chỉ role 'user'
            config=generation_config # Truyền System Instruction qua config
        )
        return response.text
        
    except Exception as e:
        # Xử lý lỗi Rate Limit (429) hoặc lỗi khác.
        print(f"LỖI API: {e}. Đang tạm dừng 60 giây và thử lại chunk này.")
        time.sleep(60)
        # Thử lại lần 2
        try:
            response = client.models.generate_content(
                model=MODEL_NAME, 
                contents=contents, 
                config=generation_config
            )
            return response.text
        except Exception as retry_e:
            print(f"LỖI THỬ LẠI: Không thể xử lý chunk này. Lưu chunk trống. {retry_e}")
            return "" 

# ==============================================================================
#                 QUÁ TRÌNH LẶP VÀ XỬ LÝ CHÍNH
# ==============================================================================

def process_all_chunks(chunks, client):
    # Đảm bảo client đã khởi tạo thành công
    if client is None:
        print("\nKhông thể tiến hành xử lý. Client chưa được khởi tạo thành công.")
        return []

    corrected_pages = []
    total_chunks = len(chunks)
    start_time = time.time()

    print(f"\n--- BẮT ĐẦU XỬ LÝ VỚI {MODEL_NAME} ---")
    print(f"Tổng số chunks: {total_chunks}. Độ trễ giữa các yêu cầu: {DELAY_SECONDS} giây.")

    for i, chunk in enumerate(chunks):
        print(f"\n[{time.strftime('%H:%M:%S')}] Đang xử lý chunk {i+1}/{total_chunks}...")
        
        corrected_text = correct_ocr_error(chunk, client)
        corrected_pages.append(corrected_text)
        
        # --- KIỂM SOÁT TỐC ĐỘ (RPM) ---
        if i < total_chunks - 1:
            print(f"   Hoàn thành chunk {i+1}. Tạm dừng {DELAY_SECONDS} giây.")
            time.sleep(DELAY_SECONDS)
        else:
            print(f"   Hoàn thành chunk cuối cùng.")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\n--- HOÀN TẤT XỬ LÝ ---")
    print(f"Tổng thời gian xử lý: {elapsed_time:.2f} giây (khoảng {elapsed_time/60:.2f} phút).")
    return corrected_pages

if client: 
    corrected_results = process_all_chunks(text_chunks, client) 

    # Ghép nối và lưu lại
    final_corrected_document = "\n\n[--- KẾT THÚC CHUNK ---]\n\n".join(corrected_results)
    with open(OUTPUT_FILENAME, "w", encoding="utf-8") as f:
        f.write(final_corrected_document)
        
    print(f"\nDữ liệu đã sửa lỗi đã được lưu vào tệp: {OUTPUT_FILENAME}")


--- BẮT ĐẦU XỬ LÝ VỚI gemini-2.5-flash ---
Tổng số chunks: 45. Độ trễ giữa các yêu cầu: 5 giây.

[11:36:15] Đang xử lý chunk 1/45...
   ✅ Hoàn thành chunk 1. Tạm dừng 5 giây.

[11:37:40] Đang xử lý chunk 2/45...
   ✅ Hoàn thành chunk 2. Tạm dừng 5 giây.

[11:40:00] Đang xử lý chunk 3/45...
   ✅ Hoàn thành chunk 3. Tạm dừng 5 giây.

[11:42:02] Đang xử lý chunk 4/45...
   ✅ Hoàn thành chunk 4. Tạm dừng 5 giây.

[11:43:13] Đang xử lý chunk 5/45...
   ✅ Hoàn thành chunk 5. Tạm dừng 5 giây.

[11:45:21] Đang xử lý chunk 6/45...
   ✅ Hoàn thành chunk 6. Tạm dừng 5 giây.

[11:47:31] Đang xử lý chunk 7/45...
   ✅ Hoàn thành chunk 7. Tạm dừng 5 giây.

[11:49:01] Đang xử lý chunk 8/45...
   ✅ Hoàn thành chunk 8. Tạm dừng 5 giây.

[11:52:04] Đang xử lý chunk 9/45...
   ✅ Hoàn thành chunk 9. Tạm dừng 5 giây.

[11:54:18] Đang xử lý chunk 10/45...
   ✅ Hoàn thành chunk 10. Tạm dừng 5 giây.

[11:56:08] Đang xử lý chunk 11/45...
   ✅ Hoàn thành chunk 11. Tạm dừng 5 giây.

[11:58:27] Đang xử lý chunk 1

In [ ]:
import os

OUTPUT_FILENAME = 'tai_lieu_da_sua_loi_final.md'
FINAL_CLEAN_FILENAME = 'tai_lieu_hoan_chinh.md'
CHUNK_SEPARATOR = "\n\n[--- KẾT THÚC CHUNK ---]\n\n"

def clean_final_document(input_filename, output_filename, separator):
    """
    Đọc tệp đầu vào, xóa tất cả các dấu phân cách chunk, 
    và lưu vào tệp đầu ra mới.
    """
    if not os.path.exists(input_filename):
        print(f"❌ Lỗi: Không tìm thấy tệp đầu vào '{input_filename}'.")
        return

    print(f"Đang đọc dữ liệu từ '{input_filename}'...")
    
    with open(input_filename, 'r', encoding='utf-8') as f:
        full_content = f.read()

    # 1. Xóa tất cả các chuỗi phân cách CHUNK_SEPARATOR
    # Thay thế chuỗi phân cách (bao gồm cả dòng mới) bằng MỘT LẦN xuống dòng hoặc không gian
    # Trong trường hợp này, vì đây là văn bản đã được AI format (có xuống dòng), 
    # việc thay thế bằng một khoảng trắng hoặc một lần xuống dòng là hợp lý nhất.
    # Ta sẽ thay thế bằng hai lần xuống dòng ('\n\n') để đảm bảo đoạn văn bản được nối lại 
    # có một khoảng cách đoạn hợp lý, trừ khi AI đã tự xử lý. 
    # Tuy nhiên, cách an toàn nhất là xóa hoàn toàn nó đi.
    
    # Ta thay thế chuỗi phân cách bằng MỘT KHOẢNG TRẮNG, để đảm bảo nếu từ cuối chunk 1 
    # và từ đầu chunk 2 bị dính vào nhau sau khi xóa phân cách, thì sẽ có khoảng trắng.
    # Tuy nhiên, vì AI đã trả về markdown, nên ta chỉ cần thay thế bằng một chuỗi rỗng:
    clean_content = full_content.replace(separator, "") 

    # 2. Xử lý các khoảng trắng/xuống dòng thừa sau khi nối (nếu có)
    # Loại bỏ các chuỗi khoảng trắng lặp lại do quá trình nối
    clean_content = re.sub(r'[ \t]+', ' ', clean_content)
    # Loại bỏ các chuỗi xuống dòng thừa sau khi xóa phân cách (chỉ giữ lại \n\n)
    clean_content = re.sub(r'\n{3,}', '\n\n', clean_content)
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(clean_content)

    print(f"Đã làm sạch và lưu tài liệu vào tệp: '{output_filename}'")

# --- THỰC THI HÀM LÀM SẠCH ---
# Chạy hàm này sau khi file OUTPUT_FILENAME (file đã có dấu phân cách) đã được tạo.
clean_final_document(OUTPUT_FILENAME, FINAL_CLEAN_FILENAME, CHUNK_SEPARATOR)

Đang đọc dữ liệu từ 'tai_lieu_da_sua_loi_final.md'...
✅ Đã làm sạch và lưu tài liệu vào tệp: 'tai_lieu_hoan_chinh.md'
